In [15]:
import tensorflow as tf
from tensorflow import keras

import numpy as np
import matplotlib.pyplot as plt
import h5py
import os,time,sys

from importlib import reload
import fidle

from myutils.dataset import  read_dataset
from myutils.models import get_model_v1

In [29]:
enhanced_dir = './data'

dataset_name  = 'set-24x24-L'
batch_size    = 64
epochs        = 10
scale         = 1
fit_verbosity = 1

In [28]:
x_train,y_train,x_test,y_test = read_dataset(enhanced_dir, dataset_name)
x_train,y_train, x_test,y_test = fidle.utils.rescale_dataset(x_train,y_train,x_test,y_test, scale=scale)

(932, 24, 24, 1) (932,)
Datasets have been shuffled.
Datasets have been resized with a factor  1


In [21]:
def get_model_v1(lx,ly,lz):
    
    model = keras.models.Sequential()
    
    model.add( keras.layers.Conv2D(96, (3,3), activation='relu', input_shape=(lx,ly,lz)))
    model.add( keras.layers.MaxPooling2D((2, 2)))
    model.add( keras.layers.Dropout(0.2))

    model.add( keras.layers.Conv2D(192, (3, 3), activation='relu'))
    model.add( keras.layers.MaxPooling2D((2, 2)))
    model.add( keras.layers.Dropout(0.2))

    model.add( keras.layers.Flatten()) 
    model.add( keras.layers.Dense(1500, activation='relu'))
    model.add( keras.layers.Dropout(0.5))

    model.add( keras.layers.Dense(43, activation='softmax'))
    return model

In [22]:
(n,lx,ly,lz) = x_train.shape
print("Images of the dataset have this folowing shape : ",(lx,ly,lz))

model = get_model_v1(lx,ly,lz)

model.summary()

model.compile(optimizer = 'adam',
              loss      = 'sparse_categorical_crossentropy',
              metrics   = ['accuracy'])


Images of the dataset have this folowing shape :  (24, 24, 1)
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 22, 22, 96)        960       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 11, 11, 96)       0         
 2D)                                                             
                                                                 
 dropout_3 (Dropout)         (None, 11, 11, 96)        0         
                                                                 
 conv2d_3 (Conv2D)           (None, 9, 9, 192)         166080    
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 4, 4, 192)        0         
 2D)                                                             
                                                          

In [23]:
history = model.fit(  x_train, y_train,
                      batch_size      = batch_size,
                      epochs          = epochs,
                      verbose         = fit_verbosity,
                      validation_data = (x_test, y_test))

Epoch 1/10
15/15 [==============================] - 2s 113ms/step - loss: 3.1725 - accuracy: 0.1384 - val_loss: 2.9746 - val_accuracy: 0.1459
Epoch 2/10
15/15 [==============================] - 2s 120ms/step - loss: 2.9177 - accuracy: 0.1470 - val_loss: 2.8872 - val_accuracy: 0.1760
Epoch 3/10
15/15 [==============================] - 2s 144ms/step - loss: 2.8033 - accuracy: 0.1685 - val_loss: 2.7822 - val_accuracy: 0.2017
Epoch 4/10
15/15 [==============================] - 2s 148ms/step - loss: 2.6612 - accuracy: 0.2049 - val_loss: 2.5590 - val_accuracy: 0.2747
Epoch 5/10
15/15 [==============================] - 2s 140ms/step - loss: 2.4026 - accuracy: 0.3004 - val_loss: 2.4120 - val_accuracy: 0.3090
Epoch 6/10
15/15 [==============================] - 2s 102ms/step - loss: 2.2522 - accuracy: 0.3487 - val_loss: 2.1508 - val_accuracy: 0.3691
Epoch 7/10
15/15 [==============================] - 2s 103ms/step - loss: 2.0888 - accuracy: 0.3884 - val_loss: 2.0945 - val_accuracy: 0.3734
Epoch 

In [24]:
max_val_accuracy = max(history.history["val_accuracy"])
print("Max validation accuracy is : {:.4f}".format(max_val_accuracy))

Max validation accuracy is : 0.5064


In [25]:
score = model.evaluate(x_test, y_test, verbose=0)

print('Test loss      : {:5.4f}'.format(score[0]))
print('Test accuracy  : {:5.4f}'.format(score[1]))

Test loss      : 1.7192
Test accuracy  : 0.5064
